In [1]:
import os
import utils
import config
import recordgen
import preprocess
from TLT import *


# Load the model from the file system if it is stored.
if os.path.exists(config.model_store):
    print("Loading a model from the file system.")
    weights = utils.load_model_weights()
    model = TwoLevelTransformerModel(weights)
else:
    print("Compiling a new model.")
    _, word_repr = utils.load_wordvecs(config.word2vec_file_path, load_embs=True)
    model = TwoLevelTransformerModel(word_repr=word_repr)

/home/mario/.local/share/virtualenvs/topic-segmentation-ivNeB2ii/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Loading a model from the file system.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(16, 14, 20)]       0           []                               
                                                                                                  
 lambda_1 (Lambda)              (16, 14, 20)         0           ['input_1[0][0]']                
                                                                                                  
 lambda (Lambda)                (16, 14, 20)         0           ['input_1[0][0]']                
                                                                                                  
 embedding_2 (Embedding)        (16, 14, 20, 10)     140         ['lambda_1[0][0]']               
                                                        

In [ ]:
input_dir = "data/wiki_727K/train/"
tfrecord_tmpl = os.path.join(input_dir, config.tfrecord_tmpl)

#preprocess.preprocess_wiki(input_dir)
#recordgen.record_gen(input_dir, tfrecord_tmpl)

batch_count = recordgen.get_batch_count(tfrecord_tmpl)
print("Number of batches =", batch_count)

ds = recordgen.get_dataset(tfrecord_tmpl)
model.fit(ds, steps_per_epoch=batch_count)

In [10]:
import importlib, TLT, config
importlib.reload(config)
importlib.reload(TLT)
importlib.reload(recordgen)
from TLT import *

#model = TwoLevelTransformerModel(word_repr=word_repr)

input_dir = "data/overfit"
tfrecord_tmpl = os.path.join(input_dir, config.tfrecord_tmpl)

#preprocess.preprocess_wiki(input_dir)
#recordgen.record_gen(input_dir, tfrecord_tmpl)

batch_count = recordgen.get_batch_count(tfrecord_tmpl)
print("Number of batches =", batch_count)

ds = recordgen.get_dataset(tfrecord_tmpl)
model.fit(ds, steps_per_epoch=batch_count, epochs=config.epochs)

Number of batches = 178
Epoch 1/2
178/178 [==============================] - 25s 142ms/step - loss: 0.0038 - binary_accuracy: 0.9989
Epoch 2/2
178/178 [==============================] - 26s 148ms/step - loss: 0.0040 - binary_accuracy: 0.9988


In [2]:
import numpy as np
input_dir = "data/"
tfrecord_tmpl = os.path.join(input_dir, config.tfrecord_tmpl)

ds = recordgen.get_dataset(tfrecord_tmpl)
total = 0
for (f, l), _ in zip(ds, range(500)):
    w = model.predict(f)
    x = np.where(w >= 0.5, 1, 0)
    misses = (x != l.numpy()).sum()
    total += misses
    print(misses)

1/1 [==============================] - 3s 3s/step
0
1/1 [==============================] - 0s 50ms/step
0
1/1 [==============================] - 0s 47ms/step
0
1/1 [==============================] - 0s 45ms/step
0
1/1 [==============================] - 0s 50ms/step
0
1/1 [==============================] - 0s 46ms/step
0
1/1 [==============================] - 0s 53ms/step
0
1/1 [==============================] - 0s 49ms/step
0
1/1 [==============================] - 0s 53ms/step
0
1/1 [==============================] - 0s 55ms/step
0
1/1 [==============================] - 0s 70ms/step
0
1/1 [==============================] - 0s 71ms/step
0
1/1 [==============================] - 0s 49ms/step
3
1/1 [==============================] - 0s 46ms/step
0
1/1 [==============================] - 0s 44ms/step
0
1/1 [==============================] - 0s 59ms/step
0
1/1 [==============================] - 0s 60ms/step
0
1/1 [==============================] - 0s 59ms/step
0
1/1 [=========================

In [3]:
print(total)
#utils.store_model_weights(model)

187


In [ ]:
import os
import config
import preprocess
import recordgen

input_dir = "data/"
tfrecord_tmpl = os.path.join(input_dir, config.tfrecord_tmpl)

import utils
words = utils.load_wordvecs(config.word2vec_file_path)
word_inv = dict((i, w) for i, w in enumerate(words))

preprocess.preprocess_wiki(input_dir)
recordgen.record_gen(input_dir, tfrecord_tmpl)

# ds = recordgen.input_fn(tfrecord_tmpl)
# for snippet_batch, labels_batch in ds:
#     for snippet in snippet_batch:
#         for segment in snippet:
#             for token in segment:
#                 print(word_inv[int(token)], end=' ')
#             print()
#     break

In [ ]:
import importlib, TLT
importlib.reload(TLT)
from TLT import *

if False:
    import os, utils, recordgen, preprocess
    input_dir = "data/"
    tfrecord_tmpl = os.path.join(input_dir, config.tfrecord_tmpl)
    words, word_repr = utils.load_wordvecs(config.word2vec_file_path, load_embs=True)
    word_inv = dict((i, w) for i, w in enumerate(words))

model = TwoLevelTransformerModel(word_repr)
ds = recordgen.get_dataset(tfrecord_tmpl)

for f, l in ds:
    model.fit(f, l)
    lp = model.predict(f)
    for sntok, snemb in zip(f, lp):
        for i, (sgtok, sgemb) in enumerate(zip(sntok, snemb)):
            print(i + 1, end=' ')
            for tok, emb in zip(sgtok, sgemb):
                print(word_inv[int(tok)], end=' ')
                if not np.all(word_repr[int(tok)] == emb[config.positional_embeddings_length:]):
                    raise RuntimeError("wtf")
            print()
        print('\n -------------------- \n')

In [ ]:
import importlib, TLT
importlib.reload(TLT)
from TLT import *

model = TwoLevelTransformerModel(word_repr)
ds = recordgen.get_dataset(tfrecord_tmpl)

for f, l in ds:
    lp = model.predict(f, steps=1)
    for sntok, snemb in zip(f, lp):
        for i, (sgtok, sgemb) in enumerate(zip(sntok, snemb)):
            print(i + 1, sgemb)
        print('\n -------------------- \n')